In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# Load Data

In [2]:
def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [7]:
store_files = [
    "../synthetic_data/RCT_0_5.h5",
    "../synthetic_data/RCT_0_05.h5",
    "../synthetic_data/e_X.h5",
    "../synthetic_data/e_X_U.h5",
    "../synthetic_data/e_X_no_overlap.h5",
    "../synthetic_data/e_X_info_censor.h5",
    "../synthetic_data/e_X_U_info_censor.h5",
    "../synthetic_data/e_X_no_overlap_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in range(1, 11):
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            continue
    experiment_setups[base_name] = scenario_dict

In [8]:
experiment_setups['RCT_0_5']['scenario_1']['dataset'].head()

,id,observed_time,event,W,X1,X2,X3,X4,X5,U1,U2,T0,T1,T,C
0,0,0.054267,1,0,0.135488,0.887852,0.932606,0.445568,0.388236,0.151609,0.205535,0.054267,0.061394,0.054267,1.803019
1,1,0.732630,1,1,0.257596,0.657368,0.492617,0.964238,0.800984,0.597208,0.255785,0.228566,0.732630,0.732630,1.689546
2,2,0.162856,1,1,0.455205,0.801058,0.041718,0.769458,0.003171,0.370382,0.223214,0.176016,0.162856,0.162856,1.256329
3,3,0.050340,1,1,0.292809,0.610914,0.913027,0.300115,0.248599,0.038464,0.409829,0.381909,0.050340,0.050340,1.241777
4,4,0.524607,1,0,0.666392,0.987533,0.468270,0.123287,0.916031,0.342961,0.791330,0.524607,1.121968,0.524607,1.516613


In [10]:
experiment_repeat_setups = pd.read_csv("../synthetic_data/idx_split.csv").set_index("idx")
experiment_repeat_setups

,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
idx,,,,,,,,,,
0,47390,5618,14210,46970,4203,16369,24535,45204,45725,45885
1,38566,46218,39045,7253,22759,34401,28889,38471,45822,37471
2,32814,20226,40012,4854,27351,39165,25359,14516,25717,29860
3,41393,39492,27153,19041,33009,19822,21243,41228,955,23901
4,12564,17823,48976,18458,22756,28169,45851,36620,29824,12711
...,...,...,...,...,...,...,...,...,...,...
49995,15948,39245,30779,48178,45056,4892,528,7486,31042,38267
49996,11102,29624,40779,3136,45904,41903,45682,36621,33204,38070
49997,16338,8986,19293,35651,10172,17947,38843,18310,2765,12581


# EXPERIMENT CONSTANTS

In [11]:
NUM_REPEATS_TO_INCLUDE = 10  # max 10
NUM_TRAINING_DATA_POINTS = 5000 # max 45000
TEST_SIZE = 5000

In [12]:
output_pickle_path = f"results/s_learner_num_repeats_{NUM_REPEATS_TO_INCLUDE}_train_size_{NUM_TRAINING_DATA_POINTS}.pkl"

# Run Experiments

In [18]:
import sys
import os

# Add the parent directory of "notebooks" to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from models_causal_impute_meta.meta_learners import SLearner

import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [9]:
def prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list):
    split_results = {}

    for rand_idx in random_idx_col_list:
        random_idx = experiment_repeat_setups[rand_idx].values
        test_ids = random_idx[-TEST_SIZE:]
        train_ids = random_idx[:min(NUM_TRAINING_DATA_POINTS, len(random_idx) - TEST_SIZE)]

        X_cols = [c for c in dataset_df.columns if c.startswith("X") and c[1:].isdigit()]
        
        train_df = dataset_df[dataset_df['id'].isin(train_ids)]
        test_df = dataset_df[dataset_df['id'].isin(test_ids)]

        X_train = train_df[X_cols].to_numpy()
        W_train = train_df["W"].to_numpy()
        Y_train = train_df[["observed_time", "event"]].to_numpy()

        X_test = test_df[X_cols].to_numpy()
        W_test = test_df["W"].to_numpy()
        Y_test = test_df[["observed_time", "event"]].to_numpy()

        cate_test_true = (test_df["T1"] - test_df["T0"]).to_numpy()

        split_results[rand_idx] = (X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true)

    return split_results

In [ ]:
random_idx_col_list = experiment_repeat_setups.columns.to_list()[:NUM_REPEATS_TO_INCLUDE]

print("Output results path:", output_pickle_path)

base_regressors = ['ridge', 'lasso', 'rf', 'gbr', 'xgb']

results_dict = {}

for setup_name, setup_dict in tqdm(experiment_setups.items(), desc="Experiment Setups"):

    results_dict[setup_name] = {}

    for scenario_key in tqdm(setup_dict):
        dataset_df = setup_dict[scenario_key]["dataset"]
        split_dict = prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list)

        # Initialize results dictionary for this setup and scenario
        results_dict[setup_name][scenario_key] = {}

        # For each base model, we will run the SLearner
        for base_model in tqdm(base_regressors):
            # print(f"Running {base_model} for {setup_name} - {scenario_key}")
            
            # Store placeholder for later population
            results_dict[setup_name][scenario_key][base_model] = {}
            
            start_time = time.time()

            for rand_idx in random_idx_col_list:
                X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true = split_dict[rand_idx]

                ##################################
                # TODO: Imputation Step
                Y_train = Y_train[:, 0]  # observed_time
                Y_test  = Y_test[:, 0]   # observed_time
                ##################################
                
                
                learner = SLearner(base_model_name=base_model)
                learner.fit(X_train, W_train, Y_train)
                mse_test, cate_test_pred, ate_test_pred = learner.evaluate(X_test, cate_test_true, W_test)

                # Save results
                results_dict[setup_name][scenario_key][base_model][rand_idx] = {
                    "cate_true": cate_test_true,
                    "cate_pred": cate_test_pred,
                    "ate_true": cate_test_true.mean(),
                    "ate_pred": ate_test_pred,
                    "cate_mse": mse_test,
                    "ate_bias": ate_test_pred - cate_test_true.mean(),
                }

            end_time = time.time()

            # Save results to the setup dictionary
            results_dict[setup_name][scenario_key][base_model]["average"] = {
                "mean_cate_mse": np.mean([results_dict[setup_name][scenario_key][base_model][rand_idx]["cate_mse"]
                                        for rand_idx in random_idx_col_list]),
                "std_cate_mse": np.std([results_dict[setup_name][scenario_key][base_model][rand_idx]["cate_mse"]
                                        for rand_idx in random_idx_col_list]),
                "mean_ate_pred": np.mean([results_dict[setup_name][scenario_key][base_model][rand_idx]["ate_pred"]
                                        for rand_idx in random_idx_col_list]),
                "std_ate_pred": np.std([results_dict[setup_name][scenario_key][base_model][rand_idx]["ate_pred"]
                                        for rand_idx in random_idx_col_list]),
                "mean_ate_true": np.mean([results_dict[setup_name][scenario_key][base_model][rand_idx]["ate_true"]
                                        for rand_idx in random_idx_col_list]),
                "std_ate_true": np.std([results_dict[setup_name][scenario_key][base_model][rand_idx]["ate_true"]
                                        for rand_idx in random_idx_col_list]),
                "mean_ate_bias": np.mean([results_dict[setup_name][scenario_key][base_model][rand_idx]["ate_bias"]
                                        for rand_idx in random_idx_col_list]),
                "std_ate_bias": np.std([results_dict[setup_name][scenario_key][base_model][rand_idx]["ate_bias"]
                                        for rand_idx in random_idx_col_list]),
                "runtime": (end_time - start_time) / len(random_idx_col_list)
            }

            # Save progress to disk
            with open(output_pickle_path, "wb") as f:
                pickle.dump(results_dict, f)
            
            # break
        # break
    # break
            


Output results path: results/s_learner_num_repeats_10_train_size_5000.pkl


Experiment Setups:   0%|          | 0/8 [00:00<?, ?it/s]





100%|██████████| 5/5 [03:18<00:00, 39.63s/it]






100%|██████████| 5/5 [03:49<00:00, 45.90s/it]






100%|██████████| 5/5 [04:41<00:00, 56.38s/it]






100%|██████████| 5/5 [03:33<00:00, 42.76s/it]






Experiment Setups:  12%|█▎        | 1/8 [19:54<2:19:23, 1194.79s/it]





100%|██████████| 5/5 [03:48<00:00, 45.63s/it]






100%|██████████| 5/5 [03:47<00:00, 45.56s/it]






100%|██████████| 5/5 [05:16<00:00, 63.20s/it] 






100%|██████████| 5/5 [04:17<00:00, 51.52s/it]






Experiment Setups:  25%|██▌       | 2/8 [41:08<2:04:06, 1241.08s/it]





100%|██████████| 5/5 [03:09<00:00, 37.94s/it]






100%|██████████| 5/5 [03:10<00:00, 38.19s/it]






100%|██████████| 5/5 [07:21<00:00, 88.22s/it] 






100%|██████████| 5/5 [04:16<00:00, 51.25s/it]






Experiment Setups:  38%|███▊      | 3/8 [1:03:00<1:46:07, 1273.58s/it]





100%|██████████| 5/5 [03:56<00:00, 47.25s/it]






100%|██████████| 5/5 [03:21<00:00, 4

In [11]:
def summarize_experiment_results(results_dict):
    records = []

    for setup_name, setup_dict in results_dict.items():
        for scenario_key in setup_dict:
            row = {
                ("setup_name", ""): setup_name,
                ("scenario_key", ""): scenario_key
            }

            for base_model in setup_dict[scenario_key]:
                avg_result = setup_dict[scenario_key].get(base_model, {}).get("average", {})
                mean_mse = avg_result.get("mean_cate_mse", np.nan)
                std_mse = avg_result.get("std_cate_mse", np.nan)
                mean_ate_pred = avg_result.get("mean_ate_pred", np.nan)
                std_ate_pred = avg_result.get("std_ate_pred", np.nan)
                mean_ate_true = avg_result.get("mean_ate_true", np.nan)
                std_ate_true = avg_result.get("std_ate_true", np.nan)
                mean_ate_bias = avg_result.get("mean_ate_bias", np.nan)
                std_ate_bias = avg_result.get("std_ate_bias", np.nan)
                runtime = avg_result.get("runtime", np.nan)

                row[(base_model, "CATE_MSE")] = f"{mean_mse:.3f} ± {std_mse:.3f}" if not pd.isna(mean_mse) else np.nan
                row[(base_model, "ATE_pred")] = f"{mean_ate_pred:.3f} ± {std_ate_pred:.3f}" if not pd.isna(mean_ate_pred) else np.nan
                row[(base_model, "ATE_true")] = f"{mean_ate_true:.3f} ± {std_ate_true:.3f}" if not pd.isna(mean_ate_true) else np.nan
                row[(base_model, "ATE_bias")] = f"{mean_ate_bias:.3f} ± {std_ate_bias:.3f}" if not pd.isna(mean_ate_bias) else np.nan
                row[(base_model, "runtime [s]")] = round(runtime) if not pd.isna(runtime) else np.nan

            records.append(row)

    df = pd.DataFrame.from_records(records)
    df.columns = pd.MultiIndex.from_tuples(df.columns)
    return df


In [12]:
print("S-Learner Results")
summary_df = summarize_experiment_results(results_dict)
summary_df

S-Learner Results


setup_name scenario_key           ridge                  \
                                                   CATE_MSE        ATE_pred   
0                      RCT_0_5   scenario_1   0.693 ± 0.068   0.048 ± 0.007   
1                      RCT_0_5   scenario_2  13.701 ± 1.700   0.005 ± 0.013   
2                      RCT_0_5   scenario_5  14.879 ± 0.370   0.457 ± 0.076   
3                      RCT_0_5   scenario_8  16.969 ± 0.305   0.081 ± 0.066   
4                      RCT_0_5   scenario_9  43.261 ± 4.003  -0.177 ± 0.004   
5                     RCT_0_05   scenario_1   0.691 ± 0.068   0.070 ± 0.029   
6                     RCT_0_05   scenario_2  13.703 ± 1.702  -0.003 ± 0.029   
7                     RCT_0_05   scenario_5  14.965 ± 0.311   0.418 ± 0.264   
8                     RCT_0_05   scenario_8  16.949 ± 0.292   0.099 ± 0.094   
9                     RCT_0_05   scenario_9  43.249 ± 4.018  -0.170 ± 0.010   
10                         e_X   scenario_1   0.696 ± 0.068   0.034 ± 0.009   
11                         e_X   scenario_2  13.697 ± 1.700   0.016 ± 0.018   
12                         e_X   scenario_5  14.873 ± 0.394   0.485 ± 0.103   
13                         e_X   scenario_8  16.987 ± 0.272   0.064 ± 0.045   
14                         e_X   scenario_9  43.263 ± 4.000  -0.179 ± 0.004   
15                       e_X_U   scenario_1   1.212 ± 0.136   0.017 ± 0.008   
16                       e_X_U   scenario_2   5.707 ± 0.835   0.038 ± 0.015   
17                       e_X_U   scenario_5  15.243 ± 0.266   0.359 ± 0.132   
18                       e_X_U   scenario_8  17.431 ± 0.415   0.030 ± 0.043   
19                       e_X_U   scenario_9  74.842 ± 7.721  -0.183 ± 0.003   
20              e_X_no_overlap   scenario_1   0.697 ± 0.068   0.028 ± 0.007   
21              e_X_no_overlap   scenario_2  13.703 ± 1.696  -0.001 ± 0.019   
22              e_X_no_overlap   scenario_5  14.898 ± 0.347   0.432 ± 0.092   
23              e_X_no_overlap   scenario_8  16.931 ± 0.282   0.106 ± 0.050   
24              e_X_no_overlap   scenario_9  43.266 ± 4.001  -0.180 ± 0.006   
25             e_X_info_censor   scenario_1   0.691 ± 0.067   0.065 ± 0.011   
26             e_X_info_censor   scenario_2  13.697 ± 1.699   0.016 ± 0.023   
27             e_X_info_censor   scenario_5  15.497 ± 0.348  -0.089 ± 0.046   
28             e_X_info_censor   scenario_8  17.210 ± 0.274  -0.079 ± 0.055   
29             e_X_info_censor   scenario_9  43.052 ± 4.006  -0.052 ± 0.026   
30           e_X_U_info_censor   scenario_1   1.200 ± 0.134   0.069 ± 0.015   
31           e_X_U_info_censor   scenario_2   5.706 ± 0.834   0.052 ± 0.021   
32           e_X_U_info_censor   scenario_5  15.756 ± 0.229  -0.078 ± 0.059   
33           e_X_U_info_censor   scenario_8  17.585 ± 0.449  -0.071 ± 0.041   
34           e_X_U_info_censor   scenario_9  74.625 ± 7.693  -0.081 ± 0.028   
35  e_X_no_overlap_info_censor   scenario_1   0.691 ± 0.067   0.064 ± 0.010   
36  e_X_no_overlap_info_censor   scenario_2  13.705 ± 1.695  -0.008 ± 0.024   
37  e_X_no_overlap_info_censor   scenario_5  15.433 ± 0.405  -0.047 ± 0.059   
38  e_X_no_overlap_info_censor   scenario_8  17.226 ± 0.326  -0.087 ± 0.047   
39  e_X_no_overlap_info_censor   scenario_9  43.049 ± 4.007  -0.051 ± 0.037   

                                                        lasso                  \
         ATE_true        ATE_bias runtime [s]        CATE_MSE        ATE_pred   
0   0.126 ± 0.011  -0.078 ± 0.014           0   0.693 ± 0.068   0.047 ± 0.007   
1   0.165 ± 0.057  -0.160 ± 0.064           0  13.701 ± 1.700   0.003 ± 0.009   
2   0.751 ± 0.060  -0.294 ± 0.079           0  14.879 ± 0.369   0.457 ± 0.076   
3   0.775 ± 0.042  -0.695 ± 0.094           0  16.976 ± 0.302   0.075 ± 0.059   
4   0.729 ± 0.086  -0.906 ± 0.087           0  43.258 ± 4.003  -0.176 ± 0.004   
5   0.126 ± 0.011  -0.057 ± 0.029           0   0.691 ± 0.068   0.066 ± 0.029   
6   0.165 ± 0.057  -0.167 ± 0.058           0  13.702